# Build data

This script builds the necessary data files for homework 1. We work from the raw data and save intermediate data for future analysis.

## Preliminary loading and packages

In [1]:
!pip -q install rpy2
%load_ext rpy2.ipython

In [3]:
%%R
options(repos = c(CRAN = "https://cloud.r-project.org"))

library(tidyverse) 
library(ggplot2) 
library(dplyr)
library(lubridate)
library(stringr)
library(readxl)
library(data.table)
library(gdata) 
library(scales)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


data.table 1.14.8 using 2 threads (see ?getDTthreads).  Latest news: r-datatable.com

Attaching package: ‘data.table’

The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose


Attaching package: ‘gdata’

The following objects are masked from ‘package:data.table’:

    first, last

The following objects are masked from ‘package:dplyr’:

    combine, first, last, starts_with

The following object is masked from ‘package:purrr’:

    keep

The following object is masked from ‘package:tidyr’:

    starts_with

The following object is masked from ‘package:stats’:

    nobs

The following object is masked from ‘package:utils’:

    object.size

The following object is masked from ‘package:base’:

    startsWith


Attaching package: ‘scales’

The following 

## Read enrollment and contract data

In [4]:
%%R
  read_contract <- function(path) {
    read_csv(
      path,
      skip = 1,
      col_names = c(
        "contractid","planid","org_type","plan_type","partd","snp","eghp",
        "org_name","org_marketing_name","plan_name","parent_org","contract_date"
      ),
      col_types = cols(
        contractid = col_character(),
        planid     = col_double(),
        org_type   = col_character(),
        plan_type  = col_character(),
        partd      = col_character(),
        snp        = col_character(),
        eghp       = col_character(),
        org_name   = col_character(),
        org_marketing_name = col_character(),
        plan_name  = col_character(),
        parent_org = col_character(),
        contract_date = col_character()
      ),
      show_col_types = FALSE,
      progress = FALSE
    )
  }

  read_enroll <- function(path) {
    read_csv(
      path,
      skip = 1,
      col_names = c("contractid","planid","ssa","fips","state","county","enrollment"),
      col_types = cols(
        contractid = col_character(),
        planid     = col_double(),
        ssa        = col_double(),
        fips       = col_double(),
        state      = col_character(),
        county     = col_character(),
        enrollment = col_double()
      ),
      na = "*",
      show_col_types = FALSE,
      progress = FALSE
    )
  }

  # One-month loader --------------------------------------------------------
  load_month <- function(m, y) {
    c_path <- paste0("../ma-data/ma/enrollment/Extracted Data/CPSC_Contract_Info_", y, "_", m, ".csv")
    e_path <- paste0("../ma-data/ma/enrollment/Extracted Data/CPSC_Enrollment_Info_", y, "_", m, ".csv")

    contract.info <- read_contract(c_path) %>%
      distinct(contractid, planid, .keep_all = TRUE)   

    enroll.info <- read_enroll(e_path)

    contract.info %>%
      left_join(enroll.info, by = c("contractid","planid")) %>%
      mutate(month = as.integer(m), year = y)
  }

In [6]:
%%R

  monthlist <- sprintf("%02d", 1:12)
  y <- 2018

  # Read all months, then tidy once ----------------------------------------

  plan.year <- map_dfr(monthlist, ~ load_month(.x, y)) %>%
    arrange(contractid, planid, state, county, month) %>%
    group_by(state, county) %>%
    fill(fips, .direction = "downup") %>%                
    ungroup() %>%
    group_by(contractid, planid) %>%
    fill(plan_type, partd, snp, eghp, plan_name, .direction = "downup") %>%
    ungroup() %>%
    group_by(contractid) %>%
    fill(org_type, org_name, org_marketing_name, parent_org, .direction = "downup") %>%
    ungroup()


  # Collapse to yearly panel ------------------------------------------------
  final.plans <- plan.year %>%
    group_by(contractid, planid, fips, year) %>%
    arrange(month, .by_group = TRUE) %>%
    summarize(
      n_nonmiss        = sum(!is.na(enrollment)),
      avg_enrollment   = ifelse(n_nonmiss > 0, mean(enrollment, na.rm = TRUE), NA_real_),
      sd_enrollment    = ifelse(n_nonmiss > 1, sd(enrollment, na.rm = TRUE), NA_real_),
      min_enrollment   = ifelse(n_nonmiss > 0, min(enrollment, na.rm = TRUE), NA_real_),
      max_enrollment   = ifelse(n_nonmiss > 0, max(enrollment, na.rm = TRUE), NA_real_),
      first_enrollment = ifelse(n_nonmiss > 0, first(na.omit(enrollment)), NA_real_),
      last_enrollment  = ifelse(n_nonmiss > 0,  last(na.omit(enrollment)), NA_real_),
      state            = last(state),
      county           = last(county),
      org_type         = last(org_type),
      plan_type        = last(plan_type),
      partd            = last(partd),
      snp              = last(snp),
      eghp             = last(eghp),
      org_name         = last(org_name),
      org_marketing_name = last(org_marketing_name),
      plan_name        = last(plan_name),
      parent_org       = last(parent_org),
      contract_date    = last(contract_date),
      year             = last(year),
      .groups = "drop"
    )



In addition: There were 12 warnings (use warnings() to see them)


In [7]:
%%R

write_csv(final.plans,"data/output/plan_data.csv")

Error: Cannot open file for writing:
* 'data/output/plan_data.csv'


RInterpreterError: Failed to parse and evaluate line '\nwrite_csv(final.plans,"data/output/plan_data.csv")\n'.
R error message: "Error: Cannot open file for writing:\n* 'data/output/plan_data.csv'"

## Read service area data

In [12]:
%%R
  read_service_area <- function(path) {
    read_csv(
      path, skip = 1,
      col_names = c(
        "contractid","org_name","org_type","plan_type","partial","eghp",
        "ssa","fips","county","state","notes"
      ),
      col_types = cols(
        contractid = col_character(),
        org_name   = col_character(),
        org_type   = col_character(),
        plan_type  = col_character(),
        partial    = col_logical(),
        eghp       = col_character(),
        ssa        = col_double(),
        fips       = col_double(),
        county     = col_character(),
        state      = col_character(),
        notes      = col_character()
      ),
      na = "*",
      show_col_types = FALSE,
      progress = FALSE
    )
  }

  # One-month loader --------------------------------------------------------
  load_month_sa <- function(m, y) {
    path <- paste0("../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_",y, "_", m, ".csv")
    
    read_service_area(path) %>%
      mutate(month = as.integer(m), year = y)
  }


In [13]:
%%R

  monthlist <- sprintf("%02d", 1:12)
  y <- 2018

  service.year <- map_dfr(monthlist, ~ load_month_sa(.x, y))

  # Ensure stable order before fills
  service.year <- service.year %>%
    arrange(contractid, fips, state, county, month)

  # Fill missing identifiers/labels
  service.year <- service.year %>%
    group_by(state, county) %>%
    fill(fips, .direction = "downup") %>%
    ungroup() %>%
    group_by(contractid) %>%
    fill(plan_type, partial, eghp, org_type, org_name, .direction = "downup") %>%
    ungroup()

  # Collapse to yearly: one row per contract × county (fips) × year --------
  final.service.area <- service.year %>%
    group_by(contractid, fips, year) %>%
    arrange(month, .by_group = TRUE) %>%
    summarize(
      state     = last(state),
      county    = last(county),
      org_name  = last(org_name),
      org_type  = last(org_type),
      plan_type = last(plan_type),
      partial   = last(partial),
      eghp      = last(eghp),
      ssa       = last(ssa),
      notes     = last(notes),
      .groups = "drop"
    )


In addition: There were 12 warnings (use warnings() to see them)


In [14]:
%%R

write_csv(final.service.area,"data/output/service_area.csv")